# perparing the data

In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import sklearn
import seaborn as sbe
import os 
import math 
import shutil
from glob import glob
from numpy import array
from imutils import paths
from tensorflow.keras import models,layers
import cv2

In [5]:
training_ds=tf.data.Dataset.list_files("Teeth DataSet\\Teeth_Dataset\\Training\\*\\*",shuffle=False)
testing_ds=tf.data.Dataset.list_files("Teeth DataSet\\Teeth_Dataset\\Testing\\*\\*",shuffle=False)
validation_ds=tf.data.Dataset.list_files("Teeth DataSet\\Teeth_Dataset\\Validation\\*\\*",shuffle=False)
for f in training_ds.take(4):
    print(f.numpy())
for f in testing_ds.take(4):
    print(f.numpy())
for f in validation_ds.take(4):
    print(f.numpy())


b'Teeth DataSet\\Teeth_Dataset\\Training\\CaS\\a_100_0_1462.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Training\\CaS\\a_100_0_4424.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Training\\CaS\\a_100_0_5215.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Training\\CaS\\a_100_0_5539.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Testing\\CaS\\a_100.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Testing\\CaS\\a_101_0_1635.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Testing\\CaS\\a_101_0_1711.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Testing\\CaS\\a_101_0_4911.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Validation\\CaS\\a_100_0_982.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Validation\\CaS\\a_101_0_2726.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Validation\\CaS\\a_101_0_5413.jpg'
b'Teeth DataSet\\Teeth_Dataset\\Validation\\CaS\\a_102_0_5905.jpg'


In [6]:
class_name=["CaS","CoS","Gum","MC","OC","OLP","OT"]
training_ds = training_ds.shuffle(200)
testing_ds = testing_ds.shuffle(200)
validation_ds = validation_ds.shuffle(200)
image_train_count=len(training_ds)
print(image_train_count)
print(len(testing_ds))
print(len(validation_ds))


3087
1028
1028


In [7]:
#s="Teeth DataSet\\Teeth_Dataset\\Training\\MC"
#s.split("\\")[-1]
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]


In [8]:
training_ds.map(get_label)

<_MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [10]:
def process_image(file_path):
    label=get_label(file_path)
    img = tf.io.read_file(file_path)  # Read the image file
    img = tf.image.decode_jpeg(img, channels=3)  # Decode as JPEG
    img = tf.image.resize(img, [256, 256])  # Resize to 256x256
    return img, label  # Return image and label as a tuple


#training_ds = training_ds.map(process_image)

In [11]:
img, label = process_image("Teeth DataSet\\Teeth_Dataset\\Training\\CaS\\a_100_0_1462.jpg")
img.numpy()[:2]

array([[[252., 217., 197.],
        [252., 217., 197.],
        [252., 217., 197.],
        ...,
        [245., 244., 249.],
        [245., 244., 249.],
        [245., 244., 249.]],

       [[252., 217., 197.],
        [252., 217., 197.],
        [252., 217., 197.],
        ...,
        [245., 244., 249.],
        [245., 244., 249.],
        [245., 244., 249.]]], dtype=float32)

In [12]:
training_ds=training_ds.map(process_image)
testing_ds=testing_ds.map(process_image)
validation_ds=validation_ds.map(process_image)
for img,label in training_ds.take(2):
    print(img)
    

tf.Tensor(
[[[255. 254. 255.]
  [255. 254. 255.]
  [255. 254. 255.]
  ...
  [223. 132. 127.]
  [223. 132. 127.]
  [223. 132. 127.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [223. 132. 127.]
  [224. 133. 128.]
  [224. 133. 128.]]

 [[255. 255. 251.]
  [255. 255. 251.]
  [255. 255. 251.]
  ...
  [227. 134. 127.]
  [227. 134. 127.]
  [228. 135. 128.]]

 ...

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [252. 141. 157.]
  [252. 140. 156.]
  [251. 140. 156.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [253. 145. 160.]
  [254. 143. 159.]
  [252. 144. 159.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 150. 165.]
  [255. 149. 164.]
  [255. 149. 164.]]], shape=(256, 256, 3), dtype=float32)
tf.Tensor(
[[[255. 234. 221.]
  [255. 235. 221.]
  [255. 235. 221.]
  ...
  [237. 131. 143.]
  [237. 131. 143.]
  [237. 131. 143.]]

 [[255. 227. 215.]
  [254. 228. 213.]
  [253. 230. 216.]
  ...
  [237. 131. 143.]

In [13]:
for image, label in training_ds.take(1):
    print("****",image)
    print("****",label)

**** tf.Tensor(
[[[177. 105.  91.]
  [177. 105.  91.]
  [177. 105.  91.]
  ...
  [161. 124.  97.]
  [159. 124.  96.]
  [157. 122.  94.]]

 [[176. 104.  90.]
  [176. 104.  90.]
  [176. 104.  90.]
  ...
  [161. 124.  97.]
  [158. 123.  95.]
  [157. 122.  94.]]

 [[174. 105.  90.]
  [174. 105.  90.]
  [176. 104.  90.]
  ...
  [161. 124.  97.]
  [158. 123.  95.]
  [157. 122.  94.]]

 ...

 [[195. 113. 102.]
  [207. 125. 114.]
  [211. 129. 117.]
  ...
  [190. 121.  90.]
  [189. 120.  89.]
  [188. 119.  88.]]

 [[201. 119. 108.]
  [208. 126. 115.]
  [208. 126. 114.]
  ...
  [193. 124.  93.]
  [192. 123.  92.]
  [191. 122.  91.]]

 [[205. 123. 112.]
  [209. 127. 116.]
  [205. 123. 111.]
  ...
  [195. 126.  95.]
  [194. 125.  94.]
  [194. 125.  94.]]], shape=(256, 256, 3), dtype=float32)
**** tf.Tensor(b'CaS', shape=(), dtype=string)


In [14]:
def scale(image, label):
    return image/255, label

In [15]:
training_ds=training_ds.map(scale)
testing_ds=testing_ds.map(scale)
validation_ds=validation_ds.map(scale)

In [16]:
# Extracting images (x_train) and labels (y_train) as numpy arrays
x_train = []
y_train = []

for img, label in training_ds.as_numpy_iterator():
    x_train.append(img)
    y_train.append(label)

# Convert lists to numpy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)

print(x_train.shape, y_train.shape)  # Check the shapes of the data
#print(type(y_train[0]))

(3087, 256, 256, 3) (3087,)


In [17]:
# Extracting images (x_train) and labels (y_train) as numpy arrays
x_test = []
y_test = []

for img, label in testing_ds.as_numpy_iterator():
    x_test.append(img)
    y_test.append(label)

# Convert lists to numpy arrays
x_test = np.array(x_test)
y_test = np.array(y_test)

print(x_test.shape, y_test.shape)  # Check the shapes of the data
#print(type(y_train[0]))

(1028, 256, 256, 3) (1028,)


In [18]:
# Extracting images (x_train) and labels (y_train) as numpy arrays
x_val = []
y_val = []

for img, label in validation_ds.as_numpy_iterator():
    x_val.append(img)
    y_val.append(label)

# Convert lists to numpy arrays
x_val = np.array(x_val)
y_val = np.array(y_val)

print(x_val.shape, y_val.shape)  # Check the shapes of the data
#print(type(y_train[0]))

(1028, 256, 256, 3) (1028,)


In [19]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_val = x_val.astype('float32')

In [20]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit the encoder and transform string labels to integer labels
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)
y_val = label_encoder.fit_transform(y_val)
print(y_train)

[0 0 0 ... 6 6 6]


In [21]:
print(x_train.shape)
print(y_train.shape)
y_train = y_train.reshape(-1,)
print([y_train])

(3087, 256, 256, 3)
(3087,)
[array([0, 0, 0, ..., 6, 6, 6], dtype=int64)]


here we prepare the data to be used in the modeling process

# Preparing the model

In [22]:
cnn = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(256, 256, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(7, activation='softmax')
])

c:\Users\shahd\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
cnn.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

In [25]:
cnn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 246016)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │    15,745,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           455 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,294,807 (180.42 MB)

 Trainable params: 15,764,935 (60.14 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 31,529,872 (120.28 MB)

In [24]:
cnn.fit(x_train, y_train, epochs=10)

Epoch 1/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 41s 409ms/step - accuracy: 0.1934 - loss: 3.5111
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 39s 406ms/step - accuracy: 0.3638 - loss: 1.6998
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 39s 399ms/step - accuracy: 0.5284 - loss: 1.3989
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 38s 390ms/step - accuracy: 0.7733 - loss: 0.6997
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 38s 388ms/step - accuracy: 0.9468 - loss: 0.2330
Epoch 6/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 49s 507ms/step - accuracy: 0.9899 - loss: 0.0709
Epoch 7/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 49s 504ms/step - accuracy: 0.9971 - loss: 0.0156
Epoch 8/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 41s 424ms/step - accuracy: 0.9993 - loss: 0.0052
Epoch 9/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 53s 548ms/step - accuracy: 1.0000 - loss: 0.0016
Epoch 10/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 40s 414ms/step - accuracy: 1.0000 - loss: 9.3422e-04
